# <a id='unsupervised-learning'></a> 2. [**Apprentissage non supervisé**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/2_unsupervised_learning.ipynb#model-selection-and-evaluation)</br>([*Unsupervised learning*](https://scikit-learn.org/stable/unsupervised_learning.html#unsupervised-learning))

# 2.4. [**Regroupement double (_Biclustering_)**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/2_4_biclustering.ipynb#biclustering)<br/>([_Biclustering_](https://scikit-learn.org/stable/modules/biclustering.html#biclustering))

# Sommaire

- **Volume** : 7 pages, 3 exemples, 3 papiers
- 2.4.1. [**Regroupement conjoint spectral (_Co-Clustering_ spectral)**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/2_4_biclustering.ipynb#spectral-co-clustering)<br/>([_Spectral Co-Clustering_](https://scikit-learn.org/stable/modules/biclustering.html#spectral-co-clustering))
- 2.4.2. [**Regroupement double spectral (_Biclustering_ spectral)**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/2_4_biclustering.ipynb#spectral-biclustering)<br/>([_Spectral Biclustering_](https://scikit-learn.org/stable/modules/biclustering.html#spectral-biclustering))
- 2.4.3. [**Évaluation du regroupement double**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/2_4_biclustering.ipynb#biclustering-evaluation)<br/>([_Biclustering evaluation_](https://scikit-learn.org/stable/modules/biclustering.html#biclustering-evaluation))

# <a id='biclustering'></a> 2.4. Regroupement Double (_Biclustering_)

Le regroupement double peut être effectué à l'aide du module `sklearn.cluster.bicluster`. Les algorithmes de regroupement double regroupent simultanément les lignes et les colonnes d'une matrice de données. Ces regroupements de lignes et de colonnes sont appelés des biclusters. Chacun détermine une sous-matrice de la matrice de données originale avec certaines propriétés souhaitées.

Par exemple, étant donné une matrice de forme `(10, 10)`, un bicluster possible avec trois lignes et deux colonnes induit une sous-matrice de forme `(3, 2)` :

In [1]:
import numpy as np
data = np.arange(100).reshape(10, 10)
lignes = np.array([0, 2, 3])[:, np.newaxis]
colonnes = np.array([1, 2])
data[lignes, colonnes]
# array([[ 1,  2],
#        [21, 22],
#        [31, 32]])

array([[ 1,  2],
       [21, 22],
       [31, 32]])

À des fins de visualisation, étant donné un bicluster, les lignes et les colonnes de la matrice de données peuvent être réarrangées pour rendre le bicluster contigu.

Les algorithmes diffèrent dans la façon dont ils définissent les biclusters. Certains des types courants comprennent :
- valeurs constantes, lignes constantes ou colonnes constantes
- valeurs exceptionnellement élevées ou faibles
- sous-matrices à faible variance
- lignes ou colonnes corrélées

Les algorithmes diffèrent également dans la façon dont les lignes et les colonnes peuvent être attribuées aux biclusters, ce qui conduit à différentes structures de biclusters. Des structures diagonales en blocs ou en damier se produisent lorsque les lignes et les colonnes sont divisées en partitions.

Si chaque ligne et chaque colonne appartient exactement à un bicluster, alors le réarrangement des lignes et des colonnes de la matrice de données révèle les biclusters sur la diagonale. Voici un exemple de cette structure où les biclusters ont des valeurs moyennes plus élevées que les autres lignes et colonnes :

<div style="background-color: white; color: black; text-align: center;">
  <img
    src="https://scikit-learn.org/stable/_images/sphx_glr_plot_spectral_coclustering_003.png"
    alt="Un exemple de biclusters formés en partitionnant les lignes et les colonnes"
    style="max-width: 50%; height: auto;"/>
    <p>Un exemple de biclusters formés en partitionnant les lignes et les colonnes.</p>
</div>

Dans le cas du damier, chaque ligne appartient à tous les groupes de colonnes, et chaque colonne appartient à tous les groupes de lignes. Voici un exemple de cette structure où la variance des valeurs au sein de chaque bicluster est faible :

<div style="background-color: white; color: black; text-align: center;">
  <img
    src="https://scikit-learn.org/stable/_images/sphx_glr_plot_spectral_biclustering_003.png"
    alt="Un exemple de biclusters formés en partitionnant les lignes et les colonnes"
    style="max-width: 50%; height: auto;"/>
    <p>Un exemple de biclusters en damier.</p>
</div>

Après avoir ajusté un modèle, l'appartenance des groupes de lignes et de colonnes peut être trouvée dans les attributs `rows_` et `columns_`. `rows_[i]` est un vecteur binaire avec des entrées non nulles correspondant aux lignes appartenant au bicluster `i`. De même, `columns_[i]` indique quelles colonnes appartiennent au bicluster `i`.

Certains modèles ont également des attributs `row_labels_` et `column_labels_`. Ces modèles divisent les lignes et les colonnes, comme dans les structures de biclusters en blocs diagonaux et en damier.

> **Remarque :** Le regroupement double a de nombreux autres noms dans différents domaines, notamment le co-regroupement (_co-clustering_), le regroupement à deux modes (_two-mode clustering_), le regroupement bidirectionnel (_two-way clustering_), le regroupement en blocs (_block clustering_), le regroupement bidirectionnel couplé (_coupled two-way clustering_), etc. Les noms de certains algorithmes, tels que l'algorithme de co-regroupement spectral (_Spectral Co-Clustering_), reflètent ces noms alternatifs.

## <a id='spectral-co-clustering'></a> 2.4.1. Regroupement Conjoint Spectral

L'algorithme [**`SpectralCoclustering`**](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.SpectralCoclustering.html) trouve des biclusters avec des valeurs plus élevées que celles des autres lignes et colonnes correspondantes. Chaque ligne et chaque colonne appartiennent exactement à un bicluster, donc le réarrangement des lignes et des colonnes pour rendre les partitions contiguës révèle ces valeurs élevées le long de la diagonale :

**Remarque :** L'algorithme traite la matrice de données d'entrée comme un graphe biparti : les lignes et les colonnes de la matrice correspondent aux deux ensembles de sommets, et chaque entrée correspond à une arête entre une ligne et une colonne. L'algorithme approxime la coupe normalisée de ce graphe pour trouver des sous-graphes importants.

### 2.4.1.1. Formulation Mathématique

Une solution approximative à la coupe normalisée optimale peut être trouvée via la décomposition en valeurs propres généralisée du Laplacien du graphe. Habituellement, cela signifierait travailler directement avec la matrice du Laplacien. Si la matrice de données originale $A$ a une forme $m \times n$, le Laplacien pour le graphe biparti correspondant a une forme $(m + n) \times (m + n)$. Cependant, dans ce cas, il est possible de travailler directement avec $A$, qui est plus petite et plus efficace.

La matrice d'entrée $A$ est prétraitée comme suit :

$$A_n = R^{-1/2} A C^{-1/2}$$

Où $R$ est la matrice diagonale avec l'entrée $i$ égale à $\sum_{j} A_{ij}$ et $C$ est la matrice diagonale avec l'entrée $j$ égale à $\sum_{i} A_{ij}$.

La décomposition en valeurs singulières, $A_n = U \Sigma V^\top$, fournit les partitions des lignes et des colonnes de $A$. Un sous-ensemble des vecteurs singuliers gauches donne les partitions des lignes, et un sous-ensemble des vecteurs singuliers droits donne les partitions des colonnes.

Les $\ell = \lceil \log_2 k \rceil$ vecteurs singuliers, à partir du deuxième, fournissent les informations de partitionnement souhaitées. Ils sont utilisés pour former la matrice $Z$ :

$$\begin{split}Z = \begin{bmatrix} R^{-1/2} U \\\\
                    C^{-1/2} V
      \end{bmatrix}\end{split}$$

où les colonnes de $U$ sont $u_2, \dots, u_{\ell+1}$, et de même pour $V$.

Ensuite, les lignes de $Z$ sont regroupées à l'aide de [**k-means** (2.3.2)](https://scikit-learn.org/stable/modules/clustering.html#k-means). Les premières étiquettes `n_rows` fournissent le partitionnement des lignes, et les étiquettes restantes `n_columns` fournissent le partitionnement des colonnes.

#### Exemples

##### [**Une démonstration de l'algorithme de regroupement conjoint spectral**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/examples/2_4_bicluster/plot_spectral_coclustering.ipynb)<br/>([_A demo of the Spectral Co-Clustering algorithm_](https://scikit-learn.org/stable/auto_examples/bicluster/plot_spectral_coclustering.html))

Un exemple simple montrant comment générer une matrice de données avec des biclusters et comment appliquer cette méthode.

##### [**Regroupement double de documents avec l'algorithme de regroupement conjoint spectral**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/examples/2_4_bicluster/plot_bicluster_newsgroups.ipynb)<br/>([_Biclustering documents with the Spectral Co-clustering algorithm_](https://scikit-learn.org/stable/auto_examples/bicluster/plot_bicluster_newsgroups.html))

Un exemple de recherche de biclusters dans l'ensemble de données des 20 newsgroups.

#### Références

🔬 Dhillon, Inderjit S, 2001. **“Co-clustering documents and words using bipartite spectral graph partitioning”**

## <a id='spectral-biclustering'></a> 2.4.2. Regroupement Double Spectral

L'algorithme [**`SpectralBiclustering`**](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.SpectralBiclustering.html) suppose que la matrice de données d'entrée possède une structure de damier cachée. Les lignes et les colonnes d'une matrice avec cette structure peuvent être partitionnées de telle manière que les entrées de tout bicluster dans le produit cartésien des groupes de lignes et de colonnes soient approximativement constantes. Par exemple, s'il y a deux partitions de lignes et trois partitions de colonnes, chaque ligne appartiendra à trois biclusters et chaque colonne appartiendra à deux biclusters.

L'algorithme partitionne les lignes et les colonnes d'une matrice de manière à ce qu'une matrice de damier constante par bloc correspondante fournisse une bonne approximation de la matrice d'origine.

### 2.4.2.1. Formulation Mathématique

La matrice d'entrée $A$ est d'abord normalisée pour rendre le motif de damier plus évident. Trois méthodes possibles sont utilisées :

1. _Normalisation indépendante des lignes et des colonnes_, comme dans le Regroupement Conjoint Spectral. Cette méthode fait en sorte que les lignes se somment à une constante et que les colonnes se somment à une autre constante.
2. **Bistochastisation:** normalisation répétée des lignes et des colonnes jusqu'à convergence. Cette méthode fait en sorte que les lignes et les colonnes se somment à la même constante.
3. **Normalisation logarithmique:** le logarithme de la matrice de données est calculé : $L = \log A$. Ensuite, la moyenne de colonne $\overline{L_{i \cdot}}$, la moyenne de ligne $\overline{L_{\cdot j}}$ et la moyenne globale $\overline{L_{\cdot
\cdot}}$ de $L$ sont calculées. La matrice finale est calculée selon la formule

$$K_{ij} = L_{ij} - \overline{L_{i \cdot}} - \overline{L_{\cdot
j}} + \overline{L_{\cdot \cdot}}$$

Après la normalisation, les premiers vecteurs singuliers sont calculés, tout comme dans l'algorithme de Regroupement Conjoint Spectral.

Si la normalisation logarithmique a été utilisée, tous les vecteurs singuliers sont significatifs. Cependant, si la normalisation indépendante ou la bistochastisation ont été utilisées, les premiers vecteurs singuliers, $u_1$ et $v_1$, sont rejetés. À partir de maintenant, les premiers vecteurs singuliers font référence à $u_2 \dots u_{p+1}$ et $v_2 \dots v_{p+1}$, sauf dans le cas de la normalisation logarithmique.

Étant donné ces vecteurs singuliers, ils sont classés en fonction de ceux qui peuvent être le mieux approximés par un vecteur à valeurs constantes. Les approximations pour chaque vecteur sont trouvées en utilisant un k-moyennes unidimensionnel et sont évaluées à l'aide de la distance euclidienne. Un sous-ensemble des meilleurs vecteurs singuliers de gauche et de droite est sélectionné. Ensuite, les données sont projetées sur ce meilleur sous-ensemble de vecteurs singuliers et sont regroupées.

Par exemple, si $p$ vecteurs singuliers ont été calculés, les $q$ meilleurs sont trouvés comme décrit, où $q < p$. Soit $U$ la matrice dont les colonnes sont les $q$ meilleurs vecteurs singuliers de gauche, et de même ceux de droite pour $V$. Pour partitionner les lignes, les lignes de $A$ sont projetées dans un espace à $q$ dimensions : $A * V$. En traitant les $m$ lignes de cette matrice $m \times q$ comme des échantillons et en effectuant un regroupement en utilisant k-moyennes, on obtient les étiquettes des lignes. De même, en projetant les colonnes sur $A^{\top} * U$ et en regroupant cette matrice $n \times q$, on obtient les étiquettes des colonnes.

#### Exemples

##### [**Une démonstration de l'algorithme de regroupement double spectral**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/examples/2_4_bicluster/plot_spectral_biclustering.ipynb)<br/>([_A demo of the Spectral Biclustering algorithm_](https://scikit-learn.org/stable/auto_examples/bicluster/plot_spectral_biclustering.html))

Un exemple simple montrant comment générer une matrice de damier et la regrouper en biclusters.

#### Références

🔬 Kluger, Yuval, et al., 2003. [**“Spectral biclustering of microarray data: coclustering genes and conditions”**](https://genome.cshlp.org/content/13/4/703.full.pdf)

## <a id='biclustering-evaluation'></a> 2.4.3. Évaluation du Regroupement Double

Il existe deux façons d'évaluer un résultat de regroupement double : interne et externe. Les mesures internes, telles que la stabilité du cluster, reposent uniquement sur les données et les résultats eux-mêmes. Actuellement, il n'y a pas de mesures internes de bicluster dans scikit-learn. Les mesures externes font référence à une source externe d'informations, telle que la solution réelle. Lorsque l'on travaille avec des données réelles, la solution réelle est généralement inconnue, mais le regroupement de données artificielles peut être utile pour évaluer les algorithmes précisément parce que la solution réelle est connue.

Pour comparer un ensemble de biclusters trouvés à l'ensemble des biclusters réels, deux mesures de similarité sont nécessaires : une mesure de similarité pour les biclusters individuels et une façon de combiner ces similarités individuelles en un score global.

Pour comparer les biclusters individuels, plusieurs mesures ont été utilisées. Pour l'instant, seul l'indice de Jaccard est mis en œuvre :

$$J(A, B) = \frac{|A \cap B|}{|A| + |B| - |A \cap B|}$$

où $A$ et $B$ sont des biclusters, $|A \cap B|$ est le nombre d'éléments dans leur intersection. L'indice de Jaccard atteint son minimum de 0 lorsque les biclusters ne se chevauchent pas du tout et son maximum de 1 lorsqu'ils sont identiques.

Plusieurs méthodes ont été développées pour comparer deux ensembles de biclusters. Pour l'instant, seul [**`consensus_score`**](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.consensus_score.html) (Hochreiter et al., 2010) est disponible :

1. Calculer les similarités entre les paires de biclusters, un dans chaque ensemble, en utilisant l'indice de Jaccard ou une mesure similaire.
2. Assigner les biclusters d'un ensemble à un autre de manière à maximiser la somme de leurs similarités. Cette étape est effectuée à l'aide de l'algorithme hongrois.
3. La somme finale des similarités est divisée par la taille de l'ensemble le plus grand.

Le score de consensus minimum, 0, se produit lorsque toutes les paires de biclusters sont totalement différentes. Le score maximum, 1, se produit lorsque les deux ensembles sont identiques.

#### Références

🔬 Hochreiter, Bodenhofer, et al., 2010. [**“FABIA: factor analysis for bicluster acquisition”**](https://watermark.silverchair.com/bioinformatics_26_12_1520.pdf?token=AQECAHi208BE49Ooan9kkhW_Ercy7Dm3ZL_9Cf3qfKAc485ysgAAA38wggN7BgkqhkiG9w0BBwagggNsMIIDaAIBADCCA2EGCSqGSIb3DQEHATAeBglghkgBZQMEAS4wEQQMUwdjwfYrmkdfngvQAgEQgIIDMgQ85koCn6LTKeSlq7BS-F1EAoO7zYtd5XuzVE_jgm2opX4Afc7G4txytR_ReolwkNP3IgOXPGEx9P0RXaJ0IaBxDeGV9R91oeZLB4qwIsYiPuod3Trn4iH5HNeiOBJBV6obyYhNuVKOmoEJNvhy0cZQeVo6cpx-JIq7ij-udRkqmrPF7AQJjQyjDrziMZPMgI4wHY7asNMn9AgmB9zF885Upv-xpcqE5SHiYK1oiDfcYDjcyO6QJtSAs4WQ-b5Cv4Kja9m6WTwmheZ5KrswdIQK9InebEl47x_lOQPd60beEhs8YeE8pk_MISxlhgLtvG4HFwJrGVdyTetEUIQUszL_XFYvaJDA-LqD5sh95Cl5Q3_rZ-KkU1f04FHXjauzRClsluGCEFbKq7RbKWrj1Qxr0hkszVYmjnsEBqev-bJ8LV24kBf3pHTPrLMPE_xQhuvqnpyKQAiUO5uLRBfCj-zVgtBwNINi0Ef3PS7jOtSD4Y3sXRaoG3sTRKxggwfTnoBPcAaAPLIEaXs9GGUTEgQCO83f1Q5y1YBFkjgCZCOt-qXJN7kHjjVxdPN6iLlPrAHAm_cYLpnOu97L221dw5SOIUIcZylW-vRytbeU-wtvtkJOfKzzuIedYvY1LneDsTlxW8sNaBpDaxfZro0hf5HBZTG6dcxCPbtoTzRJSOl1wR3tnCXWrxHzFaRBTtphoK3zdSYoBfjqQhql5jnOPIQ6NqiUwkWX1x2jI4FTufNVGENYGeO-OPei1MdHxhaVFUzyHykO9GOaUZI6vjiN_DNj7D7iQd_1pb6MEf8NcrSGQNYHxt5axTMEm7lIh5xm3AVJOmsDZ8_3pfB5U__JI0w-FbEjZfepWbST9u6ucfgcBw49TPkQ5mlrw8a5o7OEva4mYgaUkkX9FIxvXL65YOMV5PiqEHo9QYzNL5XY7vKQnVNOIn6-YbdIscFe3WY8sBXopkf-9PWgpnj343bjOz3HsqhXwMKIlzXzqNA4ZnAWwsKwc8N0sVkB3BpGWc6-QjEURsq3eux_w3ZOebYdigQSvZ-xnwQLN7yptpEF9qvwfEGd3luOrei067cPXkYi4TIJ).